<a href="https://colab.research.google.com/github/helainytorres/python/blob/main/Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Referencia:
#https://www.datacamp.com/pt/tutorial/an-introduction-to-using-transformers-and-hugging-face

# **Tradução**

In [1]:
import pandas as pd

In [2]:
pip install transformers sentencepiece

In [3]:
from transformers import MarianTokenizer, MarianMTModel

In [4]:
#Selecione o modelo pré-treinado, obtenha o tokenizador e carregue o modelo pré-treinado

# Get the name of the model
model_name = 'Helsinki-NLP/opus-mt-en-fr'

# Get the tokenizer
tokenizer = MarianTokenizer.from_pretrained(model_name)
# Instantiate the model
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
#Adicione o token especial >>{tgt}<< na frente de cada texto de origem (inglês) com a ajuda da função a seguir.

def format_batch_texts(language_code, batch_texts):
    formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]
    return formated_bach

In [11]:
#Implemente a lógica de tradução em lote com a ajuda da função a seguir, sendo um lote uma lista de textos a serem traduzidos.

def perform_translation(batch_texts, model, tokenizer, language="fr"):

  # Prepare the text data into appropriate format for the model
  formated_batch_texts = format_batch_texts(language, batch_texts)

  # Generate translation using model
  translated = model.generate(**tokenizer(formated_batch_texts,

                                          return_tensors="pt", padding=True))

  # Convert the generated tokens indices back into text
  translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

  return translated_texts

In [19]:
# Example english texts to translate
#english_texts = ["Hello, how are you?", "I am fine, thank you."]
english_texts = ['The girl went to school. She likes to study mathematics.']

In [22]:
#Execute a tradução nas amostras anteriores de descrições

# Check the model translation from the original language (English) to French
translated_texts = perform_translation(english_texts, model, tokenizer, "fr")

# Create wrapper to properly format the text
from textwrap import TextWrapper
# Wrap text to 80 characters.
wrapper = TextWrapper(width=80)

for text in translated_texts:
  print("Original text: \n", english_texts)
  print("Translation : \n", text)
  print(print(wrapper.fill(text)))
  print("")

Original text: 
 ['The girl went to school. She likes to study mathematics.']
Translation : 
 La fille est allée à l'école. Elle aime étudier les mathématiques.
La fille est allée à l'école. Elle aime étudier les mathématiques.
None



In [18]:
translated_texts

['Bonjour, comment allez-vous ?', 'Je vais bien, merci.']

# **Zero shot rating - Cassificação de disparo zero**

O objetivo aqui é tentar classificar a categoria de cada uma das descrições anteriores, seja ela tecnologia, política, segurança ou finanças.

In [23]:
#Importar o módulo de pipeline
from transformers import pipeline

In [24]:
#Definir rótulos candidatos. Elas correspondem ao que queremos prever: tecnologia, política, negócios ou finanças
candidate_labels = ["tech", "politics", "business", "finance"]

In [25]:
#Defina o classificador com a opção multilíngue
my_classifier = pipeline("zero-shot-classification",

                           model='joeddav/xlm-roberta-large-xnli')

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


In [30]:
finance_english_text = ["""Financial markets play a crucial role in the global economy by facilitating the exchange of assets, including stocks, bonds, and commodities. Investors participate in these markets to generate returns, manage risks, and diversify their portfolios.
                           One of the most common investment strategies is value investing, where investors look for undervalued stocks that have strong fundamentals. Another popular approach is growth investing, which focuses on companies with high potential for future expansion. Meanwhile, index investing allows investors to passively track market indices, reducing risks associated with individual stock selection.
                           Risk management is essential in finance. Diversification, asset allocation, and hedging techniques help investors minimize potential losses. Additionally, macroeconomic factors such as inflation, interest rates, and economic growth significantly influence market trends and investment decisions.
                           With the rise of financial technology, algorithmic trading and cryptocurrencies have introduced new opportunities and challenges in financial markets. Investors must stay informed and adapt to changing market conditions to optimize their financial strategies.
                        """]

In [31]:
#Execute as previsões na primeira e na última descrições

#For the first description
prediction = my_classifier(finance_english_text, candidate_labels, multi_class = True)
pd.DataFrame(prediction).drop(["sequence"], axis=1)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


,labels,scores
0,"[finance, tech, business, politics]","[0.9948524236679077, 0.9874230623245239, 0.949..."


In [39]:
prediction_dict = prediction[0]
labels = prediction_dict['labels']
scores = prediction_dict['scores']

In [43]:
# Print labels and scores side-by-side using a loop and f-strings
for i in range(len(labels)):
  print(f"{labels[i]}: {scores[i]:.4f}")

finance: 0.9949
tech: 0.9874
business: 0.9499
politics: 0.1950


# **Análise de sentimento**

In [44]:
#Escolha a tarefa a ser executada e carregue o modelo correspondente. Aqui, queremos realizar a classificação de sentimentos usando o modelo básico destilado do BERT.
model_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
distil_bert_model = pipeline(task="sentiment-analysis", model=model_checkpoint)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [45]:
text_positiv = """The stock market has been performing exceptionally well this year.
                  Many investors have seen significant returns, and the economy is
                  showing strong signs of recovery. New technological innovations
                  are driving growth, and companies are reporting record profits.
                  With lower unemployment rates and increased consumer spending,
                  the financial outlook remains highly optimistic. Experts believe
                  that now is an excellent time to invest, as the market is expected
                  to continue its upward trend."""

In [46]:
text_negativ = """The recent economic downturn has severely impacted businesses
                  and investors. Stock prices have plummeted, and uncertainty
                  continues to dominate the market. High inflation rates and
                  rising interest costs are putting pressure on consumers,
                  leading to lower spending. Many companies are reporting losses,
                  and layoffs are becoming more frequent. Analysts warn that the
                  financial crisis could deepen if immediate action is not taken
                  to stabilize the economy."""

In [47]:
text_neutro = """The financial market experienced mixed results this quarter.
                While some industries showed moderate growth, others faced
                challenges due to economic uncertainties. Inflation rates
                remained steady, and central banks are closely monitoring
                market conditions. Investors are taking a cautious approach,
                waiting for further data before making major financial decisions.
                Overall, market analysts suggest that the future trend will depend
                on global economic policies and consumer behavior in the coming months."""

In [48]:
#O modelo está pronto! Vamos analisar os sentimentos subjacentes às duas últimas frases.

# Run the predictions
distil_bert_model(text_positiv)

[{'label': 'POSITIVE', 'score': 0.9994327425956726}]

In [49]:
# Run the predictions
distil_bert_model(text_negativ)

[{'label': 'NEGATIVE', 'score': 0.9959902167320251}]

In [50]:
# Run the predictions
distil_bert_model(text_neutro)

[{'label': 'NEGATIVE', 'score': 0.9904541969299316}]

# **Resposta a perguntas**

Podemos usar um modelo de pergunta-resposta da Hugging Face que fornecerá a resposta na qual estamos interessados.

Isso pode ser feito fornecendo ao modelo o contexto adequado e a pergunta para a qual estamos interessados em encontrar a resposta.

In [51]:
#Importe a classe de resposta a perguntas e o tokenizador dos transformadores
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

In [52]:
#Instanciar o modelo usando seu nome e seu tokenizador.
model_checkpoint = "deepset/roberta-base-squad2"

task = 'question-answering'
QA_model = pipeline(task, model=model_checkpoint, tokenizer=model_checkpoint)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [53]:
#Solicite o modelo fazendo a pergunta e especificando o contexto.

QA_input = {
          'question': 'when is Apple hosting an event?',
          'context': text_negativ
          }

In [55]:
#Obter o resultado do modelo

model_response = QA_model(QA_input)
print(pd.DataFrame([model_response]))

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


      score  start  end               answer
0  0.000005    141  159  \n                 


In [56]:
#Mudando a pergunta.

QA_input = {
          'question': 'Why has the economic crisis severely impacted companies and investors?',
          'context': text_negativ
          }

In [57]:
#Obter o resultado do modelo

model_response = QA_model(QA_input)
print(pd.DataFrame([model_response]))

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


      score  start  end                       answer
0  0.072162     96  123  Stock prices have plummeted
